In [1]:
import pandas as pd

In [2]:
global_confirmed = pd.read_csv('/Users/amandamarsden/Git/Covid-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
global_deaths = pd.read_csv('/Users/amandamarsden/Git/Covid-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

us_confirmed = pd.read_csv('/Users/amandamarsden/Git/Covid-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_us.csv')
us_deaths = pd.read_csv('/Users/amandamarsden/Git/Covid-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_us.csv')
us_confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,135,136,136,136,136,136,136,139,141,141
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,13,13,14,14,14,14,14,14,14,14
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1043,1068,1118,1213,1252,1298,1252,1416,1276,1307
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,51,51,53,53,53,53,54,54,54,55


In [3]:
us_confirmed_transpose = us_confirmed.transpose()
us_df = us_confirmed_transpose.drop(['UID' , 'iso2', 'iso3', 'code3',
                                       'Country_Region','Combined_Key',
                                      'Lat', 'Long_', 'FIPS', 'Admin2',
                                      'Province_State'])


In [4]:

us = []
us_date = pd.DataFrame(us_df.index,
               columns=['Date'])
for i in us_date['Date']:
    for k in us_confirmed[i]: 
        list = [i,k]
        us.append(list)
df = pd.DataFrame(us, columns = ['Date', 'Confirmed'])

In [10]:
us1=[]
for i in us_date['Date']:
    for k in us_confirmed['Combined_Key']:
        list = [k]
        us1.append(list)
df2 = pd.DataFrame(us1, columns = ['Combined_Key'])
result = pd.concat([df, df2], axis =1)

In [11]:
result

,Date,Confirmed,Combined_Key
0,1/22/20,0,"American Samoa, US"
1,1/22/20,0,"Guam, US"
2,1/22/20,0,"Northern Mariana Islands, US"
3,1/22/20,0,"Puerto Rico, US"
4,1/22/20,0,"Virgin Islands, US"
5,1/22/20,0,"Autauga, Alabama, US"
6,1/22/20,0,"Baldwin, Alabama, US"
7,1/22/20,0,"Barbour, Alabama, US"
8,1/22/20,0,"Bibb, Alabama, US"
9,1/22/20,0,"Blount, Alabama, US"
